In [ ]:
### something i found somewhere to pull an docker images for sagemaker (?)

import sagemaker

sagemaker.image_uris.retrieve(framework='spark',region='us-east-1',version='3.0',image_scope='inference')


In [7]:
!pwd
classpath = ":".join(sagemaker_pyspark.classpath_jars())
print(classpath)

/home/mick/MIDS/W210
/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/aws-java-sdk-sts-1.12.262.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemaker-1.12.262.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/aws-java-sdk-bundle-1.11.901.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/sagemaker-spark_2.12-spark_3.3.0-1.4.5.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/aws-java-sdk-kms-1.12.262.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/hadoop-aws-3.3.1.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/aws-java-sdk-core-1.12.262.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/aws-java-sdk-sagemakerruntime-1.12.262.jar:/usr/local/lib/python3.10/dist-packages/sagemaker_pyspark/jars/aws-java-sdk-s3-1.12.262.jar


In [1]:
import boto3

### imports for sagemaker pyspark
import sagemaker_pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from sagemaker_pyspark import IAMRole, classpath_jars
from sagemaker_pyspark.algorithms import KMeansSageMakerEstimator
import findspark

In [2]:
session = boto3.Session(profile_name='default')
credentials = session.get_credentials()
region = "us-east-1"

ACCESS_KEY = credentials.access_key
SECRET_KEY = credentials.secret_key
role = 'arn:aws:iam::752600073001:role/service-role/AmazonSageMaker-ExecutionRole-20230319T095240'

### Training a Model w/ SageMaker API

In [3]:
classpath = ":".join(sagemaker_pyspark.classpath_jars())
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(classpath_jars())))
spark = SparkSession.builder\
                    .config("spark.driver.extraClassPath", classpath)\
                    .config('spark.master','local[*]')\
                    .config('spark.add.name','S3app')\
                    .config('spark.jars.packages','org.apache.hadoop:hadoop-aws:3.3.4,org.apache.hadoop:hadoop-common:3.3.4')\
                    .config("spark.driver.memory", "16g") \
                    .getOrCreate()

23/04/01 23:19:01 WARN Utils: Your hostname, gator resolves to a loopback address: 127.0.1.1; using 172.31.45.57 instead (on interface eth0)
23/04/01 23:19:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mick/.ivy2/cache
The jars for the packages stored in: /home/mick/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-96651c57-be5d-4c6c-acf1-d24ba1a37417;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.hadoop#hadoop-common;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-protobuf_3_7;1.1.1 in central
	found org.apache.hadoop#hadoop-annotations;3.3.4 in central
	found org.apache.hadoop.thirdparty#hadoop-shaded-guava;1.1.1 in central
	found com.google.guava#guava;27.0-jre in central
	found com.google.guava#failureaccess;1.0 in central
	found com.google.guava#listenablefuture;9999.0-empty-to-avoid-conflict-with-guava in central
	found com.goog

23/04/01 23:19:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc=spark.sparkContext
hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.committer.name","magic")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.awsAccessKeyId", ACCESS_KEY)
hadoop_conf.set("fs.s3a.awsSecretAccessKey", SECRET_KEY)

In [ ]:
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(classpath_jars())))
SparkContext(conf=conf)

In [ ]:
# import s3fs
import configparser
import os

In [ ]:
config = configparser.ConfigParser()
aws_profile = 'default'
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get(aws_profile, "aws_access_key_id") 
access_key = config.get(aws_profile, "aws_secret_access_key")

In [ ]:
getenv()

In [5]:
processed_train_df = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_train_df.parquet")
processed_test_df = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_test_df.parquet")
processed_train_df = processed_train_df.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_test_df  = processed_test_df.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")

23/04/01 23:19:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/04/01 23:19:28 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: s3a://w210-bucket/data_wrangling/processed_train_df.parquet.
java.nio.file.AccessDeniedException: s3a://w210-bucket/data_wrangling/processed_train_df.parquet: org.apache.hadoop.fs.s3a.auth.NoAuthWithAWSException: No AWS Credentials provided by TemporaryAWSCredentialsProvider SimpleAWSCredentialsProvider EnvironmentVariableCredentialsProvider IAMInstanceCredentialsProvider : com.amazonaws.SdkClientException: Unable to load AWS credentials from environment variables (AWS_ACCESS_KEY_ID (or AWS_ACCESS_KEY) and AWS_SECRET_KEY (or AWS_SECRET_ACCESS_KEY))
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:206)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	at org.apache.hadoop.fs.

Py4JJavaError: An error occurred while calling o37.parquet.
: java.nio.file.AccessDeniedException: s3a://w210-bucket/data_wrangling/processed_train_df.parquet: org.apache.hadoop.fs.s3a.auth.NoAuthWithAWSException: No AWS Credentials provided by TemporaryAWSCredentialsProvider SimpleAWSCredentialsProvider EnvironmentVariableCredentialsProvider IAMInstanceCredentialsProvider : com.amazonaws.SdkClientException: Unable to load AWS credentials from environment variables (AWS_ACCESS_KEY_ID (or AWS_ACCESS_KEY) and AWS_SECRET_KEY (or AWS_SECRET_ACCESS_KEY))
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:206)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:170)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3289)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3185)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.getFileStatus(S3AFileSystem.java:3053)
	at org.apache.hadoop.fs.FileSystem.exi

### Building a SageMaker Model/Endpoint

In [ ]:
sm_client = boto3.client('sagemaker',
                        aws_access_key_id=ACCESS_KEY,
                        aws_secret_access_key=SECRET_KEY)
response = sm_client.list_endpoints()

len(response)

In [ ]:
# toy samples

import time
ml_model_name = "text-classification-hugging-face"
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = ml_model_name + '-model' + timestamp
endpoint_config_name = ml_model_name + '-epc' + timestamp
print(model_name)
print(endpoint_config_name)

In [ ]:
import time
ml_model_name = "test-lr-model"
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
model_name = ml_model_name + '-model' + timestamp
endpoint_config_name = ml_model_name + '-epc' + timestamp
print(model_name)
print(endpoint_config_name)

In [ ]:
# toy sample

model_data_url="763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.9-transformers4.12-cpu-py38-ubuntu20.04"

container_config = {'Image': model_data_url,
                    'Mode': 'SingleModel',
                    'Environment': {
                        'HF_MODEL_ID': 'distilbert-base-uncased-finetuned-sst-2-english',
                        'HF_TASK' : 'text-classification',
                        'SAGEMAKER_CONTAINER_LOG_LEVEL' : '20',
                        'SAGEMAKER_REGION' : 'us-east-1'
                    }
                   }

response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer=container_config,
    ExecutionRoleArn=role, 
    EnableNetworkIsolation=False
)

In [ ]:
model_data_url="763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.9-transformers4.12-cpu-py38-ubuntu20.04"

"683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sparkml-serving:3.3"

container_config = {'Image': model_data_url,
                    'Mode': 'SingleModel',
                    'Environment': {
                        'HF_MODEL_ID': 'distilbert-base-uncased-finetuned-sst-2-english',
                        'HF_TASK' : 'text-classification',
                        'SAGEMAKER_CONTAINER_LOG_LEVEL' : '20',
                        'SAGEMAKER_REGION' : 'us-east-1'
                    }
                   }

response = sm_client.create_model(
    ModelName=model_name,
    PrimaryContainer=container_config,
    ExecutionRoleArn=role, 
    EnableNetworkIsolation=False
)

In [ ]:
endpoint_config_response = sm_client.create_endpoint_config(
   EndpointConfigName=endpoint_config_name,
   ProductionVariants=[
        {
            "ModelName": model_name,
            "VariantName": "AllTraffic",
            "ServerlessConfig": {
                # Specify MemorySizeInMB and MaxConcurrency in the serverless config object
                "MemorySizeInMB": 3072,
                "MaxConcurrency": 10
            }
        }
    ]
)

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

In [ ]:
endpoint_name = "studio-lab-ep" + '-epc' + timestamp
response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)